In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dti = pd.read_csv('./DTI/biosnap.tsv', sep = '\t').rename({'#Drug': 0, 'Gene': 1}, axis = 1)

In [3]:
dti.head(2)

,0,1
0,DB00357,P05108
1,DB02721,P00325


In [4]:
len(set(dti[1].tolist()))

2325

In [5]:
len(set(dti[0].tolist() + dti[1].tolist()))

7343

In [7]:
pd.DataFrame(list(set(dti[0].tolist() + dti[1].tolist()))).rename(columns={0: "Entity_ID"}).to_csv('./DTI/entity_list.csv')

In [8]:
entity_list = pd.DataFrame(list(set(dti[0].tolist() + dti[1].tolist()))).rename(columns={0: "Entity_ID"})

In [9]:
len(dti)

15139

In [10]:
from itertools import combinations 
comb = combinations(list(entity_list.values.T[0]), 2)

In [11]:
comb = list(comb)

In [12]:
import random

In [13]:
pos = [(i[0], i[1]) for i in (dti.values)]

In [14]:
neg = list(set(comb) - set(pos))

In [15]:
comb_flipped = [(i[1], i[0]) for i in comb]

In [16]:
neg_2 = list(set(comb_flipped) - set(pos))

In [17]:
neg_2 = [(i[1], i[0]) for i in neg_2]

In [18]:
len(neg)

26948844

In [ ]:
neg_final = list(set(neg) & set(neg_2))

In [ ]:
len(neg_final)

In [ ]:
len(set(comb))

In [ ]:
random.seed(a = 1)
neg_sample = random.sample(neg_final, len(dti))

In [ ]:
len(neg_sample)

In [ ]:
df = pd.DataFrame(pos+neg_sample)
df['label'] = np.array([1]*len(pos) + [0]*len(neg_sample))

In [ ]:
df = df.rename({0:'Drug_ID', 1:'Protein_ID'}, axis = 1)

In [ ]:
def create_fold(df, x):
    test = df.sample(frac = 0.2, replace = False, random_state = x)
    train_val = df[~df.index.isin(test.index)]
    val = train_val.sample(frac = 0.125, replace = False, random_state = 1)
    train = train_val[~train_val.index.isin(val.index)]
    
    path = './DTI/fold'+str(x)
    train.reset_index(drop = True).to_csv(path + '/train.csv')
    val.reset_index(drop = True).to_csv(path + '/val.csv')
    test.reset_index(drop = True).to_csv(path + '/test.csv')
    
    return train, val, test

In [ ]:
!mkdir './DTI/fold1'

In [ ]:
train, val, test = create_fold(df, 1)